In [ ]:
%%capture
!python -m spacy download de_core_news_sm

In [1]:
import pandas as pd
import numpy as np
import re
import spacy

nlp = spacy.load('de_core_news_sm')

In [2]:
parties = ['spd', 'grüne', 'fdp', 'linke', 'afd']

In [60]:
df = pd.read_excel(f"Excel/{parties[0]} 2021.xlsx", index_col=0)

In [61]:
df.head()

,text,N Wörter,"Gelbe Zeile (A: Gehört Alleine, H: Zur Zeile darüber, R: Zur Zeile darunter)"
0,"Gelingt es uns, die Arbeit und den Wohlstand v...",28,NaN
1,Überwinden wir die wachsende Ungleichheit? Ode...,36,NaN
2,"Wird es uns gelingen, unser Leben und Wirtscha...",17,NaN
3,Nehmen wir die Gestaltung des digitalen Wandel...,29,NaN
4,Sorgen wir für gleiche Teilhabe und mehr Zusam...,27,NaN


In [62]:
for i in range(len(df.text)):
    if df.iloc[i, 2] == "H":
        df.iloc[i-1, 0] = re.sub("\.$", "", df.iloc[i-1, 0])[:-1] + " " + df.iloc[i, 0][0].lower() + df.iloc[i, 0][1:]
        df.iloc[i, 0] = np.nan
    elif df.iloc[i, 2] == "R":
        df.iloc[i+1, 0] = re.sub("\.$", "", df.iloc[i, 0]) + " " + df.iloc[i+1, 0][0].lower() + df.iloc[i+1, 0][1:]
        df.iloc[i, 0] = np.nan
    elif df.iloc[i, 2] == "F":
        df.iloc[i, 0] = np.nan
    elif df.iloc[i, 2] == "X":
        df.iloc[i, 0] = np.nan

In [64]:
df = df.dropna(subset=["text"])

In [65]:
df = df.reset_index(drop = True)

In [68]:
df.to_csv("CSV Combined/SPD 2021.csv", sep=";")

In [3]:
for party in parties:
    df = pd.read_excel(f"Excel/{party} 2021.xlsx", index_col=0)
    print("Work", party)
    
    for i in range(len(df.text)):
        if df.iloc[i, 2] == "H":
            add_sentence = df.iloc[i, 0]
            doc = nlp(add_sentence)
            if doc[0].pos_ != "Noun":
                add_sentence = df.iloc[i, 0][0].lower() + df.iloc[i, 0][1:]
            
            df.iloc[i-1, 0] = re.sub("\.$", "", str(df.iloc[i-1, 0])) + " " + add_sentence
            df.iloc[i, 0] = np.nan
        elif df.iloc[i, 2] == "R":
            add_sentence = df.iloc[i+1, 0]
            doc = nlp(add_sentence)
            if doc[0].pos_ != "Noun":
                add_sentence = df.iloc[i+1, 0][0].lower() + df.iloc[i+1, 0][1:]
                
            df.iloc[i+1, 0] = re.sub("\.$", "", df.iloc[i, 0]) + " " + add_sentence
            df.iloc[i, 0] = np.nan
        elif df.iloc[i, 2] == "F":
            df.iloc[i, 0] = np.nan
        elif df.iloc[i, 2] == "X":
            df.iloc[i, 0] = np.nan
    
    df = df.dropna(subset=["text"])
    df = df.reset_index(drop = True)
    df.text = df.text.str.lstrip()
    df.text = df.text.str.replace('?.', '?',regex=False)
    df.text = df.text.str.replace(',.', ',',regex=False)
    df.text = df.text.str.replace('!.', '!',regex=False)
    df = df.iloc[:, :1]
    df.to_csv(f"CSV Combined/{party} 2021.csv")

Work spd
Work grüne
Work fdp
Work linke
Work afd


In [4]:
df = pd.read_csv(f"CSV Combined/{parties[0]} 2021.csv", index_col=0)

In [5]:
df.head()

,text
0,"Gelingt es uns, die Arbeit und den Wohlstand v..."
1,Überwinden wir die wachsende Ungleichheit? Ode...
2,"Wird es uns gelingen, unser Leben und Wirtscha..."
3,Nehmen wir die Gestaltung des digitalen Wandel...
4,Sorgen wir für gleiche Teilhabe und mehr Zusam...
